# Extract-Transform-Load and Exploratory Data Analysis
This notebook contains all code for the prelimiatory analysis of the KDD Cup 98 datasets

In [1]:
import os
import numpy as np
import sys
os.getcwd()
proj_dir = os.path.split(os.getcwd())[0]
if proj_dir not in sys.path:
    sys.path.append(proj_dir)

## Loading the tidy dataset

In [2]:
import eda.tidy_dataset as tds
tidy = tds.TidyDataset("cup98LRN.txt", pull_stored=False)
learning = tidy.get_tidy_data()

A first look at the learning dataset

In [3]:
learning.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95412 entries, 95515 to 185114
Columns: 857 entries, ODATEDW to RFA_24_Amount_G
dtypes: category(17), datetime64[ns](2), float64(96), int64(306), object(34), uint8(402)
memory usage: 357.7+ MB


In [8]:
for c in learning.columns:
    print(c)

ODATEDW
OSOURCE
TCODE
STATE
ZIP
MAILCODE
PVASTATE
DOB
NOEXCH
RECINHSE
RECP3
RECPGVG
RECSWEEP
CLUSTER
AGE
AGEFLAG
HOMEOWNR
CHILD03
CHILD07
CHILD12
CHILD18
NUMCHLD
INCOME
GENDER
WEALTH1
HIT
MBCRAFT
MBGARDEN
MBBOOKS
MBCOLECT
MAGFAML
MAGFEM
MAGMALE
PUBGARDN
PUBCULIN
PUBHLTH
PUBDOITY
PUBNEWFN
PUBPHOTO
PUBOPP
DATASRCE
MALEMILI
MALEVET
VIETVETS
WWIIVETS
LOCALGOV
STATEGOV
FEDGOV
SOLP3
SOLIH
MAJOR
WEALTH2
GEOCODE
COLLECT1
VETERANS
BIBLE
CATLG
HOMEE
PETS
CDPLAY
STEREO
PCOWNERS
PHOTO
CRAFTS
FISHER
GARDENIN
BOATS
WALKER
KIDSTUFF
CARDS
PLATES
LIFESRC
PEPSTRFL
POP901
POP902
POP903
POP90C1
POP90C2
POP90C3
POP90C4
POP90C5
ETH1
ETH2
ETH3
ETH4
ETH5
ETH6
ETH7
ETH8
ETH9
ETH10
ETH11
ETH12
ETH13
ETH14
ETH15
ETH16
AGE901
AGE902
AGE903
AGE904
AGE905
AGE906
AGE907
CHIL1
CHIL2
CHIL3
AGEC1
AGEC2
AGEC3
AGEC4
AGEC5
AGEC6
AGEC7
CHILC1
CHILC2
CHILC3
CHILC4
CHILC5
HHAGE1
HHAGE2
HHAGE3
HHN1
HHN2
HHN3
HHN4
HHN5
HHN6
MARR1
MARR2
MARR3
MARR4
HHP1
HHP2
DW1
DW2
DW3
DW4
DW5
DW6
DW7
DW8
DW9
HV1
HV2
HV3
HV4
HU1
HU2
HU3
HU4
HU

In [32]:
raw = tidy.get_raw_data()
set(raw.RFA_2)
raw.RFA_8.isna().sum()

3511

In [25]:
learning.TARGET_B.sum()

4843

# Dummy variable cration

The column MDMAUD is the major donor matrix. It holds 4 bytes that encode the record's donation history. It will be split up into several columns of dummy variables.

In [ ]:
learning.RFA_24.unique()

In [ ]:
import cProfile
domain_spreader = tds.SymbolicFieldToDummies(learning,"RFA_24",["Recency", "Frequency", "Amount"])
cProfile.run('domain_spreader.spread()', sort='time')

In [ ]:
learning.head()

In [ ]:
import os
import numpy as np
import sys
os.getcwd()
proj_dir = os.path.split(os.getcwd())[0]
if proj_dir not in sys.path:
    sys.path.append(proj_dir)

In [ ]:
import eda.tidy_dataset as tds
tidy = tds.TidyDataset("cup98LRN.txt")

In [ ]:
raw = tidy.get_raw_data()

In [ ]:
spreader = tds.SymbolicFieldToDummies(
    raw, "RFA_24", ["Recency", "Frequency", "Amount"])
spreader.spread()